<a href="https://colab.research.google.com/github/angiekim05/hwp_to_csv/blob/main/%ED%95%9C%EA%B8%80_%EB%AC%B8%EC%84%9C_%EC%B7%A8%ED%95%A9_%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
!pip install olefile
import olefile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [209]:
class Application():
  def __init__(self,path):
    self.text = self.get_text(path)
    self.datas = self.get_datas()
    
  def print_datas(self):
    print(*self.text[:-1],sep="\n")

  def get_text(self,path):
    # 파일 불러오기기
    f = olefile.OleFileIO(path)
    encoded_txt = f.openstream("PrvText").read()
    text = encoded_txt.decode("utf-16",errors="ignore")

    # text 정제
    text = text.replace(" ","").replace("><","\t").replace(">","")
    text = text.split("\r\n")
    text = [t.replace("<","").split("\t") for t in text if len(t) > 0 and t[0] == "<"]
    text[7]=text[7][1][-4:]
    text[8]=text[8][1][-4:]
    return text

  def get_datas(self):
    datas = []
    datas.append(self.text[0][1])
    datas.append(self.text[1][1])
    datas.append(self.text[1][3])
    datas.append(self.text[2][2])  
    datas.append(self.text[2][4])
    datas += [""]+[False]*3+[""]*2

    for param in self.text[3]:
      if param[0] != "□":
        # 참가형태는 3가지 중 1개        
        if "학술대회+Workshop" in param:
          datas[5]="학술대회+Workshop"
        elif "학술대회" in param:
          datas[5]="학술대회"
        elif "Workshop" in param:
          datas[5]="Workshop"
        
        # 식사는 중복 가능능
        elif "만찬" in param:
          datas[6]=True
        elif "조식" in param:
          datas[7]=True
        elif "중식" in param:
          datas[8]=True

    for i,name in zip([1,2],["일반참가자","학생참가자"]):
      if self.text[4][i][0] != "□":
        datas[9]=name
        break
        
    for i,name in zip([1,2,3,4],["평생회원","정회원","준회원","비회원"]):
      if self.text[5][i][0] != "□":
        datas[10]=name
        break
        
    datas.append(self.text[6][2])

    money = self.text[6][4]
    if "만" in money:
      money = money[:-2]
    else:
      money = int(money[:-1].replace(",",""))//10000
    datas.append(str(money)+"만원")

    if self.text[7][0] != "□":
      datas.append("동의")
    else:
      datas.append("동의안함")
    if self.text[7][0] != "□":
      datas.append("서약함")
    else:
      datas.append("서약안함")
    return datas

In [212]:
# 파일 리스트트
import os
file_list = [name for name in os.listdir('.') if name[-3:]=="hwp"]
file_list

# 데이터 모음음
datas = []
for path in file_list:
  data = Application(path)
  data.print_datas()
  datas.append(data.datas)

['성명', '홍길동']
['소속(학생:학교)', 'ㅇㅇ대학교', '직위(학생:학위과정)', '석사과정']
['연락처', '(휴대폰)', '010-0000-0000', '(이메일)', 'sample@gmail.com']
['참가형태', '□학술대회', '식사여부(필수)', '0월0일', '□만찬', '□Workshop', '0월0일', '□조식', '□학술대회+Workshop', '□중식']
['참가구분', '□일반참가자', '□학생참가자']
['회원여부', '□평생회원', '□정회원', '□준회원', '□비회원']
['입금정보', '입금자', '홍길동', '입금액', '0만원']
□동의함
□서약함


In [215]:
import pandas as pd
cols = ['성명', '소속', '직위', '휴대폰', '이메일', '참가형태', '만찬', '조식', '중식', '참가구분', '회원여부', '입금자', '입금액',"동의","서약약"]
df = pd.DataFrame(datas, columns=cols).set_index(cols[0])
df

,소속,직위,휴대폰,이메일,참가형태,만찬,조식,중식,참가구분,회원여부,입금자,입금액,동의,서약약
성명,,,,,,,,,,,,,,
홍길동,ㅇㅇ대학교,석사과정,010-0000-0000,sample@gmail.com,,False,False,False,,,홍길동,0만원,동의안함,서약안함


In [216]:
df.to_csv("sample.csv")